<a href="https://colab.research.google.com/github/Harshpreetkaur98/Stock-Prediction/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance transformers torch datasets scikit-learn matplotlib pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import yfinance as yf
import pandas as pd

stock_data = yf.download("AAPL", start="2010-01-01", end="2023-12-01")
stock_data.reset_index(inplace=True)
stock_data.head()

[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
0,2010-01-04,6.447411,7.643214,7.660714,7.585000,7.622500,493729600
1,2010-01-05,6.458559,7.656429,7.699643,7.616071,7.664286,601904800
2,2010-01-06,6.355826,7.534643,7.686786,7.526786,7.656429,552160000
3,2010-01-07,6.344076,7.520714,7.571429,7.466071,7.562500,477131200
4,2010-01-08,6.386255,7.570714,7.571429,7.466429,7.510714,447610800


Feature Engineering

In [5]:
#Creating features
stock_data['MA_5'] = stock_data['Close'].rolling(window=5).mean()
stock_data['MA_10'] = stock_data['Close'].rolling(window=10).mean()
stock_data['RSI'] = 100 - (100 / (1 + (stock_data['Close'].diff().clip(lower=0).mean() /
                                      stock_data['Close'].diff().clip(upper=0).abs().mean())))


In [7]:
# Normalising text

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
stock_data[['Close', 'MA_5', 'MA_10', 'RSI']] = scaler.fit_transform(stock_data[['Close', 'MA_5', 'MA_10', 'RSI']])


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [8]:
# Preparing dataset for transformers

def create_sequences(data, seq_length=30):
  sequences = []
  labels = []
  for i in range(len(data) - seq_length):
    sequences.append(data[i:i + seq_length])
    labels.append(data[i + seq_length, 0])
  return sequences, labels

features = stock_data[['Close', 'MA_5', 'MA_10', 'RSI']].values
seq_length = 30
X, y =create_sequences(features, seq_length)

import numpy as np
X, y = np.array(X), np.array(y)

In [9]:
#  Splitting data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Loading a transformer Model

In [12]:
import torch
from torch.utils.data import Dataset

class StockDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

train_dataset = StockDataset(X_train, y_train)
test_dataset = StockDataset(X_test, y_test)


In [13]:
# Using a transformer for time-series

from transformers import BertModel, BertConfig

class TimeSeriesTransformer(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super(TimeSeriesTransformer, self).__init__()
    self.config = BertConfig(vocab_size=1, hidden_size=hidden_dim, num_attention_heads=4, num_hidden_layers=2)
    self.transformer = BertModel(self.config)
    self.fc = torch.nn.Linear(hidden_dim, 1)

  def forward(self, x):
    output = self.transformer(inputs_embeds=x).last_hidden_state
    return self.fc(output[:, -1, :])

model = TimeSeriesTransformer(input_dim=X_train.shape[2], hidden_dim=128)